In [33]:
import pandas as pd
import openai
import os
from docx import Document
import datetime as dt
from dotenv import load_dotenv
import concurrent.futures
# import streamlit as st 

# try:
#     openai.api_key = st.secrets["OPENAI_API_KEY"]
# except:
#     load_dotenv()
#     openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")


def create_docx(dicc, school_name, teacher_name, subject):
    """
    Creates a Word document with the given information.

    Args:
        dicc (dict): A dictionary with the topics and questions for the subject.
        school_name (str): The name of the school.
        teacher_name (str): The name of the teacher.
        subject (str): The subject that is being generated for (e.g. Math, Biology).

    Returns:
        None
    """
    
    fecha = dt.datetime.now()
    # Create a new Word document
    document = Document()

    # Set the document properties
    document.core_properties.created = fecha
    document.sections[0].header.paragraphs[0].text = f'{school_name} \t\t{fecha.day}-{fecha.month}-{fecha.year} \nTeacher: {teacher_name}'

    # Add the subtitle
    topics = ", ".join(dicc.keys())
    document.add_paragraph(f'Guía de {subject}', style='Title')
    document.add_paragraph(f'Temas revisados: {topics}', style='Subtitle')

    # Add content for each topic and question
    for topic, questions in dicc.items():
        document.add_heading(topic, level=2).bold = True
        for question, content in questions.items():
            document.add_paragraph().add_run(question).bold = True
            document.add_paragraph(content)

    # Save the document
    return document

def generate_guide(excel_file, course, subject, teacher_name, school_name):
    """
    Generates a guide with questions for students based on an Excel file.

    Args:
        excel_file (str): The path to the Excel file containing the subjects and number of questions.
        course (str): The course of the students (e.g. primaria, secundaria).
        subject (int): The subject that is being generated for (e.g. Math, Biology).
        teacher_name (str): The name of the teacher.
        school_name (str): The name of the school.

    Returns:
        None
    """
    # Read Excel file
    df = pd.read_excel(excel_file)
    df.columns = ['topic', 'number']
    
    # Initialize a list to store all the questions
    all_questions = {}
    
    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        topic = row['topic']
        num_questions = row['number']
        
        # Generate questions using OpenAI API
        questions = generate_openai_questions(topic, num_questions, course, subject)
        
        # Append questions to the list
        all_questions[topic] = questions
   
    # Create a Word document
    doc = create_docx(all_questions, school_name, teacher_name, subject)
    
    # Save the Word document
    doc.save('guide.docx')

def get_course(course):
    """
    Get the course name based on the course number.

    Args:
        course (int): Course number.

    Returns:
        str: Course name.

    Example:
        course_number = 2
        course_name = get_course(course_number)
    """
    if course == 1:
        return '1st'
    elif course == 2:
        return '2nd'
    else:
        return f'{course}rd'
    

def generate_openai_questions(topic, num_questions, course, subject):
    """
    Generates questions using the OpenAI API for a given subject.

    Args:
        topic (str): The topic of the questions. (e.g. equations, cells, etc)
        num_questions (int): The number of questions to generate.
        course (str): The course of the students (e.g. primaria, secundaria).
        subject (str): The subject that is being generated for (e.g. Math, Biology).

    Returns:
        questions: A dictionary of generated questions.
    """
    # Get configuration of messages
    messages = generate_messages(topic, course, subject)

    # Create a ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the tasks for generating comments
        results = [executor.submit(generate_question, messages) for _ in range(num_questions)]

        # Retrieve the generated comments
        questions = [result.result() for result in concurrent.futures.as_completed(results)]
       
    # Transform comments into dictionary
    questions = {f'Pregunta {_+1}' : questions[_] for _ in range(len(questions))}

    return questions

def generate_question(messages):
    """
    Generate a question based on a list of conversation messages.

    Args:
        messages (list): List of messages in the conversation.

    Returns:
        str: Generated question.

    Example:
        messages = [
            {"role": "system", "content": "System message"},
            {"role": "user", "content": "User message"}
        ]
        question = generate_question(messages)
    """
    # Call the OpenAI API to generate the response based on the conversation history
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # Extract the generated question from the response and append it to the list
    generated_question = response.choices[0].message.content
    return generated_question

def generate_messages(topic, course, subject):
    """
    Generate a list of messages for a professor designing study guides.

    Args:
        topic (str): The topic for which the multiple-choice question is generated.
        course (str): The course of the students.
        subject (str): The subject of the professor.

    Returns:
        list: List of messages for the conversation.

    Example:
        topic = "algebra"
        course = "10"
        subject = "mathematics"
    """
    #Get the course of the students
    course = get_course(course)

    # Define the initial system message providing context
    context = {
        "role": "system",
        "content": f"You are a great {subject} professor designing study guides for your {course} grade students. For the guide, you need to generate multiple-choice questions and include explanations on how to solve each question. Write in spanish"
    }
    messages = [context]

    # Prepare the user message as an order prompt for generating a question
    order_prompt = {
        'role': 'user',
        'content': f"Generate a multiple-choice question about {topic}."
    }
    messages.append(order_prompt)
    return messages

In [37]:
print(Q['Pregunta 2'])

¿Cuál es el resultado de la siguiente operación?
5 + 7 x 2 

a) 14 
b) 19 
c) 24 
d) 36 

Explicación:
Para resolver esta operación, primero debemos realizar la multiplicación (7 x 2 = 14) y luego sumar el resultado al número restante (5 + 14 = 19). Por lo tanto, la respuesta correcta es la opción b) 19.


In [31]:
print(Q['Pregunta 1'])


In [32]:
Q

{'Pregunta 1': '¿Cuál es la suma de 2 más 2?\na) 1\nb) 2\nc) 3\nd) 4\n\nExplicación: Para resolver esta pregunta, necesitamos sumar 2 más 2. La respuesta correcta es d) 4, ya que 2 + 2 es igual a 4.',
 'Pregunta 2': '¿Cuál es el resultado de la siguiente operación?\n5 + 7 x 2 \n\na) 14 \nb) 19 \nc) 24 \nd) 36 \n\nExplicación:\nPara resolver esta operación, primero debemos realizar la multiplicación (7 x 2 = 14) y luego sumar el resultado al número restante (5 + 14 = 19). Por lo tanto, la respuesta correcta es la opción b) 19.'}

In [20]:
{[f'Pregunta {_+1}'] : comments[_] for _ in range(len(comments))}

TypeError: unhashable type: 'list'

In [29]:
{f'Pregunta {_+1}' : comments[_] for _ in range(len(comments))}

{'Pregunta 1': '¿Cuánto es 2 + 2?\n\nA. 1\nB. 2\nC. 3\nD. 4\n\nRespuesta correcta: D',
 'Pregunta 2': '¿Cuál es el resultado de la suma de 5 + 7?\n\na) 10\nb) 11\nc) 12\nd) 13\n\nExplicación: Para sumar 5 + 7, comenzamos contando desde 5, y luego añadimos 7 más. Al sumarlos, obtenemos un total de 12. Por lo tanto, la respuesta correcta es la opción c.',
 'Pregunta 3': '¿Cuál es el resultado de la siguiente operación matemática?\n8 x 3 - 4 / 2 =\na) 23\nb) 28\nc) 20\nd) 22\n\nExplicación: \nPrimero se realiza la división 4 / 2 = 2.\nLuego se multiplca 8 x 3 = 24.\nFinalmente se resta 24 - 2 = 22, por lo tanto la respuesta correcta es la opción d) 22.'}

In [24]:
for _ in range(len(comments)):
    print(comments[_])

¿Cuánto es 2 + 2?

A. 1
B. 2
C. 3
D. 4

Respuesta correcta: D
¿Cuál es el resultado de la suma de 5 + 7?

a) 10
b) 11
c) 12
d) 13

Explicación: Para sumar 5 + 7, comenzamos contando desde 5, y luego añadimos 7 más. Al sumarlos, obtenemos un total de 12. Por lo tanto, la respuesta correcta es la opción c.
¿Cuál es el resultado de la siguiente operación matemática?
8 x 3 - 4 / 2 =
a) 23
b) 28
c) 20
d) 22

Explicación: 
Primero se realiza la división 4 / 2 = 2.
Luego se multiplca 8 x 3 = 24.
Finalmente se resta 24 - 2 = 22, por lo tanto la respuesta correcta es la opción d) 22.


In [2]:
def create_docx(dicc, school_name, teacher_name, subject):
    """
    Creates a Word document with the given information.

    Args:
        dicc (dict): A dictionary with the topics and questions for the subject.
        school_name (str): The name of the school.
        teacher_name (str): The name of the teacher.
        subject (str): The subject that is being generated for (e.g. Math, Biology).

    Returns:
        None
    """
    
    fecha = dt.datetime.now()
    # Create a new Word document
    document = Document()

    # Set the document properties
    document.core_properties.created = fecha
    document.sections[0].header.paragraphs[0].text = f'{school_name} \t\t{fecha.day}-{fecha.month}-{fecha.year} \nTeacher: {teacher_name}'

    # Add the subtitle
    topics = ", ".join(dicc.keys())
    document.add_paragraph(f'Guía de {subject}', style='Title')
    document.add_paragraph(f'Temas revisados: {topics}', style='Subtitle')

    # Add content for each topic and question
    for topic, questions in dicc.items():
        document.add_heading(topic, level=2).bold = True
        for question, content in questions.items():
            document.add_paragraph().add_run(question).bold = True
            document.add_paragraph(content)

    # Save the document
    return document

def generate_guide(excel_file, course, subject, teacher_name, school_name):
    """
    Generates a guide with questions for students based on an Excel file.

    Args:
        excel_file (str): The path to the Excel file containing the subjects and number of questions.
        course (str): The course of the students (e.g. primaria, secundaria).
        subject (int): The subject that is being generated for (e.g. Math, Biology).
        teacher_name (str): The name of the teacher.
        school_name (str): The name of the school.

    Returns:
        None
    """
    # Read Excel file
    df = pd.read_excel(excel_file)
    df.columns = ['topic', 'number']
    
    # Initialize a list to store all the questions
    all_questions = {}
    
    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        topic = row['topic']
        num_questions = row['number']
        
        # Generate questions using OpenAI API
        questions = generate_openai_questions(topic, num_questions, course, subject)
        
        # Append questions to the list
        all_questions[topic] = questions
   
    # Create a Word document
    doc = create_docx(all_questions, school_name, teacher_name, subject)
    
    # Save the Word document
    doc.save('guide.docx')

def generate_openai_questions(topic, num_questions, course, subject):
    """
    Generates questions using the OpenAI API for a given subject.

    Args:
        topic (str): The topic of the questions.
        num_questions (int): The number of questions to generate.
        course (str): The course of the students (e.g. primaria, secundaria).
        subject (int): The subject that is being generated for (e.g. Math, Biology).

    Returns:
        questions: A dictionary of generated questions.
    """
    # Define the initial system message providing context
    context = {
        "role": "system",
        "content": f"You are a great {subject} professor designing study guides for your {course} students. For the guide, you need to generate multiple-choice questions and include explanations on how to solve each question. Write in spanish"
    }
    messages = [context]

    questions = {}
    for _ in range(num_questions):
        # Prepare the user message as an order prompt for generating a question
        order_prompt = {
            'role': 'user',
            'content': f"Generate a multiple-choice question about {subject}."
        }
        messages.append(order_prompt)

        # Call the OpenAI API to generate the response based on the conversation history
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Extract the generated question from the response and append it to the list
        generated_question = response.choices[0].message.content
        questions[f'Pregunta {_+1}'] = generated_question

    return questions



In [3]:
generate_guide('ejemplo.xlsx', "", "", "", "")